# <center> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Identify Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score

**Считаем данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
train_df = pd.read_csv('kaggle_data/train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('kaggle_data/test_sessions.csv', index_col='session_id')

In [3]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,8,2014-01-04 08:44:50,11.0,2014-01-04 08:44:50,82.0,2014-01-04 08:45:19,68.0,2014-01-04 08:45:25,8.0,2014-01-04 08:45:25,...,2014-01-04 08:45:51,8403.0,2014-01-04 08:45:51,932.0,2014-01-04 08:45:53,3260.0,2014-01-04 08:45:53,8.0,2014-01-04 08:45:53,1845
2,111,2014-03-18 10:33:20,78.0,2014-03-18 10:33:31,151.0,2014-03-18 10:33:31,111.0,2014-03-18 10:33:31,1401.0,2014-03-18 10:33:31,...,2014-03-18 10:33:32,1375.0,2014-03-18 10:33:32,38.0,2014-03-18 10:33:32,1401.0,2014-03-18 10:33:32,97.0,2014-03-18 10:33:34,3322
3,11,2014-12-02 13:13:41,3187.0,2014-12-02 13:13:41,132.0,2014-12-02 13:13:42,496.0,2014-12-02 13:13:42,1969.0,2014-12-02 13:13:45,...,2014-12-02 13:13:45,3187.0,2014-12-02 13:13:45,82.0,2014-12-02 13:13:46,3191.0,2014-12-02 13:13:46,3184.0,2014-12-02 13:13:47,2003
4,668,2014-02-14 15:16:45,1965.0,2014-02-14 15:17:13,598.0,2014-02-14 15:20:47,1965.0,2014-02-14 15:21:13,284.0,2014-02-14 15:21:14,...,2014-02-14 15:21:14,38.0,2014-02-14 15:21:14,4451.0,2014-02-14 15:21:14,4537.0,2014-02-14 15:21:15,11.0,2014-02-14 15:21:15,1373
5,1943,2014-03-17 15:19:40,1943.0,2014-03-17 15:20:10,1943.0,2014-03-17 15:21:40,1943.0,2014-03-17 15:22:10,1943.0,2014-03-17 15:22:39,...,2014-03-17 15:22:39,1952.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:41,1943.0,2014-03-17 15:22:42,1943.0,2014-03-17 15:22:43,1737


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [4]:
train_test_df = pd.concat([train_df, test_df])

In [5]:
train_df.shape, test_df.shape, train_test_df.shape

((95319, 21), (41177, 20), (136496, 21))

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

**В обучающей выборке – 550 пользователей.**

In [6]:
train_df['user_id'].nunique()

550

**Пока для прогноза ID пользователя будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [7]:
train_sites = train_df[['site1', 'site2', 'site3','site4','site5','site6','site7',
          'site8', 'site9', 'site10', 'user_id']].fillna(0).astype('int')

In [8]:
def ures_top_sites(data, threshold=1):
    """ input dataframe
    return 
    list of top sites for users with the threshold
    dictionary user, top site's list """
    top_users_sites =[]
    dic_user_top ={}
    
    for user, values in pd.groupby(data, by = 'user_id'):
        n,m = values.shape
        sites, freq = np.unique(np.ravel(values.values), return_counts=True)
        mask = np.logical_not(np.logical_not(sites))
        sites = sites[mask]
        freq = freq[mask]
        sort_sites = sorted([(s, fr) for s,fr in zip(sites, freq)], key= lambda x: x[1], reverse =True )
        #user_site_top = sort_sites[:threshold]
        top_list = np.array([x[0] for x in  sort_sites[:threshold]])
        dic_user_top[user] = top_list
        top_users_sites.append(top_list)
    return np.unique(top_users_sites), dic_user_top

In [9]:
def tops_sites(data, threshold=3):
    """ input dataframe
    return top sites with the threshold"""
    data_ravel =np.ravel(data.drop(['user_id'], axis =1).values)
    sites, freq = np.unique(data_ravel, return_counts=True)
    mask = np.logical_not(np.logical_not(sites))
    sites = sites[mask]
    freq = freq[mask]
    mask= freq >= threshold
    sites = sites[mask]
    freq = freq[mask]
    sort_sites = sorted([(s, fr) for s,fr in zip(sites, freq)], key= lambda x: x[1], reverse =True )
    #user_site_top = sort_sites[:threshold]
    top_list = np.array([x[0] for x in  sort_sites])
    return top_list

In [10]:
def top_sites(data, threshold=50):
    """ input dataframe
    return top sites with the threshold"""
    data_ravel =np.ravel(data.drop(['user_id'], axis =1).values)
    sites, freq = np.unique(data_ravel, return_counts=True)
    mask = np.logical_not(np.logical_not(sites))
    sites = sites[mask]
    freq = freq[mask]
    sort_sites = sorted([(s, fr) for s,fr in zip(sites, freq)], key= lambda x: x[1], reverse =True )
    #user_site_top = sort_sites[:threshold]
    top_list = np.array([x[0] for x in  sort_sites[:threshold]])
    return top_list

- создадим список из 50 самых популярных сайтов

In [11]:
top_sites = top_sites(train_sites, 50)

In [12]:
len(top_sites)

50

- создадим список из сайтов которые повторяются минимум 2 раза

In [13]:
list_sites = tops_sites(train_sites, 2)

In [14]:
len(list_sites)

13988

In [15]:
def in2D(data, ar2):
    """ return the mask like data where True only the element of data is in ar2"""
    data = data.values
    n,m = data.shape
    mask = np.in1d(np.ravel(data),ar2)
    mask_2d = mask.reshape((n,m))
    return mask_2d
#np.where(mask_2d,train_sites.head(3).values, 0)

#### создадим признаки связанные с временем

In [16]:
train_test_time = train_test_df[['time1', 'time2', 'time3', 'time4','time5', 'time6',
                        'time7','time8', 'time9', 'time10']].fillna(np.datetime64('nat'))
train_test_time.head(3) 

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
1,2014-01-04 08:44:50,2014-01-04 08:44:50,2014-01-04 08:45:19,2014-01-04 08:45:25,2014-01-04 08:45:25,2014-01-04 08:45:51,2014-01-04 08:45:51,2014-01-04 08:45:53,2014-01-04 08:45:53,2014-01-04 08:45:53
2,2014-03-18 10:33:20,2014-03-18 10:33:31,2014-03-18 10:33:31,2014-03-18 10:33:31,2014-03-18 10:33:31,2014-03-18 10:33:32,2014-03-18 10:33:32,2014-03-18 10:33:32,2014-03-18 10:33:32,2014-03-18 10:33:34
3,2014-12-02 13:13:41,2014-12-02 13:13:41,2014-12-02 13:13:42,2014-12-02 13:13:42,2014-12-02 13:13:45,2014-12-02 13:13:45,2014-12-02 13:13:45,2014-12-02 13:13:46,2014-12-02 13:13:46,2014-12-02 13:13:47


In [17]:
train_test_df_sites = train_test_df[['site1', 'site2', 'site3', 'site4','site5', 
                                     'site6','site7', 'site8', 'site9', 'site10']].fillna(0).astype('int')
y = train_df['user_id']

In [18]:
mask1 = in2D(train_test_df_sites, top_sites) # маска для сайтов самых популярных у пользователей
mask2 = in2D(train_test_df_sites, list_sites)  # маска для сайтов которые встречаются в выборке больше 2 раз

**выделим сайты которые идут за сымыми популярными**

In [19]:
# оставим только сайты с частотой более 2
train_test_df_sites = np.where(mask2, train_test_df_sites.values, 0)

In [20]:
train_test_df_sites_shift = train_test_df_sites[:,1:]

In [21]:
next_top_site = np.where(mask1[:,:-1], train_test_df_sites_shift, 0) # site after top50
next_top_site.shape,  train_test_df_sites_shift.shape

((136496L, 9L), (136496L, 9L))

In [22]:
train_test_df_time = train_test_time.apply(pd.to_datetime).values

In [23]:
# оставим только сайты с частотой более 2
train_test_df_time = np.where(mask2, train_test_df_time, train_test_df_time.max())

In [24]:
# начало сессии по дням недели
day_of_week = (pd.to_datetime(np.min(train_test_df_time, axis =1))).dayofweek
day_of_week = day_of_week.reshape(train_test_df_time.shape[0],1).astype(int) 
# время начала сессии в часах
start_hour = (pd.to_datetime(np.min(train_test_df_time, axis =1))).hour
start_hour = start_hour.reshape(train_test_df_time.shape[0],1).astype(int)
# время захода на сайт из top50  
time_tops_site = np.where(mask1, train_test_df_time, train_test_df_time.max())

In [25]:
train_test_df_time = np.where(mask2, train_test_df_time, np.datetime64('nat'))

In [26]:
# время перехода между сайтами в 3 сек 
train_test_time_diff = (np.diff(train_test_df_time, axis =1)/np.timedelta64(2, 's')).round()
#test_time_diff = (np.diff(test_time, axis =1)/np.timedelta64(3, 's')).round()

In [27]:
# время перехода между сайтами из top50 and next в 2 сек 
next_time_diff_tops = np.where(mask1[:,:-1], train_test_time_diff, 0).astype(int)

In [28]:
next_time_diff_tops.max()

900

In [29]:
# время захода на сайт из top50 в часах
top_start_hour = (pd.to_datetime(np.min(time_tops_site, axis =1))).hour 
top_start_hour = top_start_hour.reshape(time_tops_site.shape[0],1).astype(int) +1
top_start_hour = np.nan_to_num(top_start_hour)

**перевод категориальных признаков в двоичные**

In [30]:
from sklearn.preprocessing import OneHotEncoder
day_of_week_encode = OneHotEncoder().fit_transform(day_of_week) # перевод категориальных признаков в двоичные
start_hour_encode = OneHotEncoder().fit_transform(start_hour)
top_start_hour_encode = OneHotEncoder().fit_transform(top_start_hour)
day_of_week_encode.shape, start_hour_encode.shape, top_start_hour_encode.shape

((136496, 7), (136496, 17), (136496, 17))

**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу train_test_df_sites – потом разделите обратно на обучающую и тестовую части.**


**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [31]:
def matrix_to_sparse_matrix (matrix):
    """переводим обычную матрицу в разреженноу матрицу 
    где 
    номер столбца это уникальное число из исходной матрицы от 1  до максимального
    значение в строке это сколько раз уникальное число встречалось в строке оригинальной матрицы"""
    import numpy as np
    from scipy.sparse import csr_matrix
        
    NMZ = np.prod(np.array(matrix.shape)) # колличество элементов в matrix
    data = np.array([1]*NMZ)
    indptr = np.arange(0, NMZ+matrix.shape[1], matrix.shape[1])
    return csr_matrix((data, matrix.reshape(-1), indptr), dtype=int)[:,1:]

In [32]:
Train_test_sparse = matrix_to_sparse_matrix(train_test_df_sites.astype(np.int64))
Train_test_sparse.shape

(136496, 21532)

In [33]:
Next_top_site_sparse = matrix_to_sparse_matrix(next_top_site.astype(np.int64))
Next_top_site_sparse.shape

(136496, 21532)

In [34]:
Next_time_tops_sparse =  matrix_to_sparse_matrix(next_time_diff_tops)
Next_time_tops_sparse.shape

(136496, 900)

**объединим все признаки сайты время начала сесии время захода на сайт из top200 and top5user**

In [35]:
from scipy.sparse import hstack
# sites + day_of_week+ start_hour + top50 hour
Train_Test= hstack((Train_test_sparse, day_of_week_encode, start_hour_encode, top_start_hour_encode ))
n1 ,m1 = Train_test_sparse.shape 
n2 ,m2 = Train_Test.shape
n1, m1, n2, m2

(136496, 21532, 136496, 21573)

In [36]:
Train_Test = hstack((Train_Test,Next_top_site_sparse))
n3, m3 =Train_Test.shape
n3, m3

(136496, 43105)

In [37]:
Train_Test = hstack((Train_Test, Next_time_tops_sparse))
n4, m4 =Train_Test.shape
n4, m4

(136496, 44005)

**разделим признаки на Train and Test**

In [38]:
X_train_spare = Train_Test.tocsr()[: 95319, :]
X_test_sparse = Train_Test.tocsr()[95319 :, :]
y = train_df['user_id'].values

In [39]:
X_train1 = X_train_spare.tocsc()[:,:m1]   # only sites
X_test1  = X_test_sparse.tocsc()[:,:m1] 
X_train2 = X_train_spare.tocsc()[:,m1:m2] # day_of_week+ start_hour + top50 hour
X_test2  = X_test_sparse.tocsc()[:,m1:m2]
X_train3 = X_train_spare.tocsc()[:,m2:m3] #  next after top50 site
X_test3  = X_test_sparse.tocsc()[:,m2:m3]
X_train4 = X_train_spare.tocsc()[:,m3:]   # time of next top50 site
X_test4  = X_test_sparse.tocsc()[:,m3:]

**Создайте объекты sklearn.linear_model.SGDClassifier с логистической функцией потерь и с hinge loss (логистическая регрессия и линейный SVM соответственно) . Остальные параметры  по умолчанию, разве что alpha=0.00007, n_jobs=-1 никогда не помешает. Обучите  модели на выборке (X_train, y_train).**

**обучим и проверим на валидации два классификатора SGDClassifier( alpha=0.00007, loss ='log') и SGDClassifier( alpha=0.00007, loss ='hinge')**

**будем последовательно обучать алгрритмы и добовлять к их Ответам новые признаки**

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train1, y, test_size=0.2, 
                                                     random_state=7, stratify=y)

In [41]:
def one_spet_fit (X_train, y_train, X_valid, y_valid, explain = "fit_SGD"):
    # Разобьем обучающую выборку на 2 части в пропорции 7/3
    
    sgd_logit = SGDClassifier( alpha=0.00007, loss ='log', random_state = 0,  n_jobs = -1)
    sgd_svm  = SGDClassifier( alpha=0.00007, loss ='hinge', random_state=0,  n_jobs = -1)
    print('fit...')
    %time sgd_logit.fit(X_train, y_train)
    %time sgd_svm.fit(X_train, y_train)
    # Сделаем прогнозы на отложенной выборке (X_valid, y_valid)
    print('predict...')
    pred_log = sgd_logit.predict(X_valid)
    pred_svm = sgd_svm.predict(X_valid)
    print (explain)
    print('log',accuracy_score(y_valid, pred_log))
    print('svm',accuracy_score(y_valid, pred_svm))
    return sgd_logit, sgd_svm

In [42]:
log_pred1, svm_pred1 = one_spet_fit (X_train, y_train, X_valid, y_valid, \
                                     explain = "sites only")

fit...
Wall time: 36.1 s
Wall time: 26.1 s
predict...
sites only
('log', 0.28960344104070501)
('svm', 0.26851657574485943)


In [43]:
def next_step (X_train, X_test, clf1, clf2):
    
    n1,_ = X_train.shape
    tr_pred1= clf1.predict(X_train).reshape(n1,1)
    tr_pred2= clf2.predict(X_train).reshape(n1,1)
    
    n2,_ = X_test.shape
    ts_pred1= clf1.predict(X_test).reshape(n2,1)
    ts_pred2= clf2.predict(X_test).reshape(n2,1)
    
    from sklearn.preprocessing import OneHotEncoder
    OHEn = OneHotEncoder()
    tr_encode1 = OHEn.fit_transform(tr_pred1)
    ts_encode1 = OHEn.transform(ts_pred1)
    tr_encode2 = OHEn.fit_transform(tr_pred2)
    ts_encode2 = OHEn.transform(ts_pred2)
    from scipy.sparse import hstack
    
    return hstack((tr_encode1, tr_encode2)), hstack((ts_encode1, ts_encode2))
    

In [44]:
X_train_step1, X_test_step1 = next_step (X_train1, X_test1, log_pred1, svm_pred1)
X_train_step1.shape, X_test_step1.shape

((95319, 1079), (41177, 1079))

In [45]:
from scipy.sparse import hstack
# step1 + day_of_week+ start_hour + top50 hour 
X_train_2 = hstack((X_train2, X_train_step1))
X_test_2  = hstack((X_test2,  X_test_step1))
X_train_2.shape, X_test_2.shape

((95319, 1120), (41177, 1120))

In [46]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_2, y, test_size=0.2, 
                                                     random_state=7, stratify=y)

In [47]:
log_pred2, svm_pred2 = one_spet_fit (X_train, y_train, X_valid, y_valid, \
                                     explain = "step1 + time")

fit...
Wall time: 34 s
Wall time: 23.1 s
predict...
sites only
('log', 0.33419009651699538)
('svm', 0.32836760386067981)


In [48]:
X_train_step2, X_test_step2 = next_step (X_train_2, X_test_2, log_pred2, svm_pred2)
X_train_step2.shape, X_test_step2.shape

((95319, 1036), (41177, 1036))

In [49]:
from scipy.sparse import hstack
# step1 + day_of_week+ start_hour + top50 hour 
X_train_3 = hstack((X_train3, X_train_step2))
X_test_3  = hstack((X_test3,  X_test_step2))
X_train_3.shape, X_test_3.shape

((95319, 22568), (41177, 22568))

In [50]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_3, y, test_size=0.2, 
                                                     random_state=7, stratify=y)

In [51]:
log_pred3, svm_pred3 = one_spet_fit (X_train, y_train, X_valid, y_valid, \
                                     explain = "step2 + next sites")

fit...
Wall time: 35.8 s
Wall time: 26.3 s
predict...
sites only
('log', 0.34232060428031891)
('svm', 0.33356063785144774)


In [52]:
X_train_step3, X_test_step3 = next_step (X_train_3, X_test_3, log_pred3, svm_pred3)
X_train_step3.shape, X_test_step3.shape

((95319, 1066), (41177, 1066))

In [53]:
from scipy.sparse import hstack
# step1 + day_of_week+ start_hour + top50 hour 
X_train_4 = hstack((X_train4, X_train_step3))
X_test_4  = hstack((X_test4,  X_test_step3))
X_train_4.shape, X_test_4.shape

((95319, 1966), (41177, 1966))

In [54]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_4, y, test_size=0.2, 
                                                     random_state=7, stratify=y)

In [55]:
log_pred4, svm_pred4 = one_spet_fit (X_train, y_train, X_valid, y_valid, \
                                     explain = "step3 + time next sites")

fit...
Wall time: 31.1 s
Wall time: 23 s
predict...
step2 + next sites
('log', 0.34420898027696184)
('svm', 0.3391733109525808)


In [86]:
X_train_spare = Train_Test_2.tocsr()[: 95319, :]
X_test_sparse = Train_Test_2.tocsr()[95319 :, :]
y = train_df['user_id'].values

In [87]:
log2, svm2 = fit_SGD (X_train_spare, y,\
             explain = "sites + day_of_week+ start_hour + top50 hour + next after top50 site")

fit...
Wall time: 41.3 s
Wall time: 31.3 s
predict...
sites + day_of_week+ start_hour + top50 hour + next after top50 site
('log', 0.35763743180864455)
('svm', 0.33004616030214018)


In [89]:
from scipy.sparse import hstack
# sites + day_of_week+ start_hour + top50 hour + 
Train_Test_3 = hstack((Train_Test_1, Next_time_tops_sparse))
Train_Test_3.shape

(136496, 22473)

In [96]:
X_train_spare = Train_Test_3.tocsr()[: 95319, :]
X_test_sparse = Train_Test_3.tocsr()[95319 :, :]
y = train_df['user_id'].values

In [91]:
log3, svm3 = fit_SGD (X_train_spare, y,\
             explain = "sites + day_of_week+ start_hour + top50 hour + time of next top50 site")

fit...
Wall time: 43.2 s
Wall time: 33.2 s
predict...
sites + day_of_week+ start_hour + top50 hour + time of next top50 site
('log', 0.36151909357952161)
('svm', 0.33648062666107148)


In [92]:
from scipy.sparse import hstack
# sites + day_of_week+ start_hour + top50 hour +next after top50 site+  time of next top50 site
Train_Test_4 = hstack((Train_Test_2, Next_time_tops_sparse))
Train_Test_4.shape

(136496, 44005)

In [93]:
X_train_spare = Train_Test_4.tocsr()[: 95319, :]
X_test_sparse = Train_Test_4.tocsr()[95319 :, :]
y = train_df['user_id'].values

In [94]:
log4, svm4 = fit_SGD (X_train_spare, y,\
             explain = "sites + day_of_week+ start_hour + top50 hour +next after top50 site+  time of next top50 site")

fit...
Wall time: 43.8 s
Wall time: 32.9 s
predict...
sites + day_of_week+ start_hour + top50 hour +next after top50 site+  time of next top50 site
('log', 0.35616869492236675)
('svm', 0.32955658134004756)


**Сделаем прогноз для тестовой выборки с помощью log3.**

In [97]:
pred_test = log3.predict(X_test_sparse)

**Запишите ответы в файл и сделайте посылку на Kaggle. Далее будет короткое Peer-Review задание, в котором надо проверить посылки друг друга на Kaggle. Поэтому дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera Username", чтоб можно было идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/leaderboard).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить, дополнительные баллы будут для тех, кто попадет в топ-10 и топ-30 по итогам этой недели.**

In [98]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [99]:
write_to_submission_file (pred_test, 'kaggle_data/[YDF&MIPT]_Coursera_Oleg67.csv')

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в финальный проект (.pdf или .ipynb).
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных 550 пользователями веб-страницах (550 csv-файлов в *train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем тьюториал по Vowpal Wabbit и попробуем его в деле – в соревновании.